In [ ]:
# Prototype 1 with 5 seeds

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from sklearn.utils.class_weight import compute_class_weight


# For random seeds-
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Load data
def load_titanic_data(num_participants):
    def _bin_age(age_series):
        bins = [-np.inf, 10, 40, np.inf]
        labels = ["Child", "Adult", "Elderly"]
        return pd.cut(age_series, bins=bins, labels=labels, right=True).astype(str).replace("nan", "Unknown")

    def _extract_title(name_series):
        titles = name_series.str.extract(" ([A-Za-z]+)\.", expand=False)
        rare_titles = {
            "Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Dona"
        }
        titles = titles.replace(list(rare_titles), "Rare")
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        return titles

    def _create_features(df):
        df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
        df["Age"] = _bin_age(df["Age"])
        df["Cabin"] = df["Cabin"].str[0].fillna("Unknown")
        df["Title"] = _extract_title(df["Name"])
        df.drop(columns=["PassengerId", "Name", "Ticket"], inplace=True)
        return df

    def vertical_partition_rotating(df, num_participants):
        partitions = [[] for _ in range(num_participants)]
        num_features = df.shape[1]

        for i, feature in enumerate(df.columns):
            participant = i % num_participants
            partitions[participant].append(feature)

        partitioned_dfs = [pd.get_dummies(df[features]) for features in partitions]
        return partitioned_dfs

    def get_partitions_and_label():
        df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
        processed_df = df.dropna(subset=["Embarked", "Fare"]).copy()
        processed_df = _create_features(processed_df)
        labels = processed_df["Survived"].values

        train_df, test_df, y_train, y_test = train_test_split(processed_df, labels, test_size=0.2, random_state=42)

        train_partitions = vertical_partition_rotating(train_df.drop(columns=["Survived"]), num_participants)
        test_partitions = vertical_partition_rotating(test_df.drop(columns=["Survived"]), num_participants)

        for i in range(len(train_partitions)):
            train_partitions[i]['Survived'] = y_train

        for i in range(len(test_partitions)):
            test_partitions[i]['Survived'] = y_test

        return train_partitions, test_partitions, y_train, y_test

    train_partitions, test_partitions, y_train, y_test = get_partitions_and_label()

    def create_tensor_datasets(partitions):
        tensor_partitions = []
        for partition in partitions:
            partition = partition.apply(pd.to_numeric, errors='coerce')
            partition = partition.fillna(0)

            for col in partition.select_dtypes(include=['bool']).columns:
                partition[col] = partition[col].astype(int)

            features = partition.drop(columns=["Survived"]).values
            labels = partition["Survived"].values.astype(np.int64)

            tensor_partition = TensorDataset(torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long))
            tensor_partitions.append(tensor_partition)
        return tensor_partitions

    train_tensor_partitions = create_tensor_datasets(train_partitions)
    test_tensor_partitions = create_tensor_datasets(test_partitions)

    return train_tensor_partitions, test_tensor_partitions, train_partitions, y_train, y_test


class GlobalModel(nn.Module):
    def __init__(self, input_sizes, hidden_sizes, output_size):
        super(GlobalModel, self).__init__()
        self.segments = nn.ModuleList()
        for input_size, hidden_size in zip(input_sizes, hidden_sizes):
            layers = [nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, output_size)]
            self.segments.append(nn.Sequential(*layers))

    def forward(self, x, active_segments):
        segment_outputs = []
        start_index = 0
        for i, segment in enumerate(self.segments):
            end_index = start_index + input_sizes[i]
            if i in active_segments:
                segment_input = x[:, start_index:end_index]
                segment_output = segment(segment_input)
                segment_outputs.append(segment_output)
            else:
                segment_outputs.append(torch.zeros(x.size(0), output_size, device=x.device))
            start_index = end_index
        combined_output = torch.mean(torch.stack(segment_outputs), dim=0)
        return combined_output


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(model, device, train_loader, optimizer, epoch, input_sizes, participant_id):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        padded_data = torch.zeros(data.size(0), sum(input_sizes)).to(device)
        start_index = sum(input_sizes[:participant_id])
        end_index = start_index + input_sizes[participant_id]
        padded_data[:, start_index:end_index] = data
        optimizer.zero_grad()
        output = model(padded_data, active_segments=[participant_id])
        loss = nn.CrossEntropyLoss(weight=class_weights)(output, target)
        loss.backward()
        optimizer.step()


def selective_exchange_gradients(models, input_sizes, hidden_sizes):
    num_models = len(models)
    param_indices = [0]
    cumulative_index = 0
    for i in range(len(hidden_sizes)):
        cumulative_index += (input_sizes[i] * hidden_sizes[i]) + hidden_sizes[i]
        param_indices.append(cumulative_index)

    for seg in range(len(hidden_sizes)):
        start = param_indices[seg]
        end = param_indices[seg + 1]
        for param_idx in range(start, end):
            grads = []
            for model in models:
                model_params = list(model.parameters())
                if param_idx < len(model_params) and model_params[param_idx].grad is not None:
                    grads.append(model_params[param_idx].grad)
            if grads:
                avg_grad = torch.stack(grads).mean(dim=0)
                for model in models:
                    model_params = list(model.parameters())
                    if param_idx < len(model_params):
                        model_params[param_idx].grad = avg_grad.clone()


def evaluate(models, device, test_loaders):
    with torch.no_grad():
        test_loss = 0
        correct = 0
        all_preds = []
        all_targets = []

        for batch_data in zip(*test_loaders):
            data_list = []
            target_list = []
            for participant_id, (data, target) in enumerate(batch_data):
                data_list.append(data)
                target_list.append(target)

            target = target_list[0].to(device)
            for t in target_list:
                assert torch.equal(t, target), "Targets are not consistent across participants"

            data_combined = torch.cat(data_list, dim=1).to(device)

            padded_data = torch.zeros(data_combined.size(0), sum(input_sizes)).to(device)
            start_index = 0
            for participant_id in range(len(input_sizes)):
                end_index = start_index + input_sizes[participant_id]
                if end_index <= data_combined.size(1):
                    padded_data[:, start_index:end_index] = data_combined[:, start_index:end_index]
                else:
                    adjusted_end_index = data_combined.size(1)
                    padded_data[:, start_index:adjusted_end_index] = data_combined[:, start_index:adjusted_end_index]
                start_index = end_index

            outputs = torch.zeros(data_combined.size(0), 2, device=device)
            for model in models:
                output = model(padded_data, active_segments=list(range(len(model.segments))))
                outputs += output
            outputs /= len(models)
            test_loss += nn.CrossEntropyLoss(reduction='sum')(outputs, target).item()
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.view(-1).cpu().numpy())
            all_targets.extend(target.view(-1).cpu().numpy())

        test_loss /= len(test_loaders[0].dataset)
        accuracy = 100. * correct / len(test_loaders[0].dataset)
        f1 = f1_score(all_targets, all_preds, average='macro')
        #print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loaders[0].dataset)} ({accuracy:.0f}%), F1-score: {f1:.4f}')

        return accuracy, f1


# Running the experiment for multiple seeds
federated_rounds = 1000
epochs_per_round = 1
hidden_size = 10  # Single hidden layer

seeds = [42, 55, 77, 101, 123]  # List of 5 different seeds
final_accuracies = []
final_f1_scores = []

for seed in seeds:
    set_random_seed(seed)
    print(f"\nRunning VFL with seed {seed}...")

    avg_accuracy = 0
    avg_f1 = 0

    for num_participants in range(2, 10):
      print(f"Running VFL with {num_participants} participants...")

      train_tensor_partitions, test_tensor_partitions, feature_partitions, y_train, y_test = load_titanic_data(num_participants)

      input_sizes = [partition.shape[1] - 1 for partition in feature_partitions]
      hidden_sizes = [10] * num_participants
      output_size = 2

      models = [GlobalModel(input_sizes, hidden_sizes, output_size).to(device) for _ in range(num_participants)]
      optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]

      class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
      class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

      best_accuracy = 0
      best_f1 = 0

      for federated_round in range(federated_rounds):
          #print(f"Federated Round {federated_round + 1}/{federated_rounds}")
          for participant_id in range(num_participants):
              train_loader = DataLoader(train_tensor_partitions[participant_id], batch_size=64, shuffle=True)
              for epoch in range(1, epochs_per_round + 1):
                train(models[participant_id], device, train_loader, optimizers[participant_id], epoch, input_sizes, participant_id)

          selective_exchange_gradients(models, input_sizes, hidden_sizes)

          test_loaders = [DataLoader(test_tensor_partitions[i], batch_size=32, shuffle=False) for i in range(num_participants)]
          accuracy, f1 = evaluate(models, device, test_loaders)

          if accuracy > best_accuracy:
              best_accuracy = accuracy
          if f1 > best_f1:
              best_f1 = f1

      print(f'Best Accuracy with {num_participants} participants: {best_accuracy:.2f}%')
      print(f'Best F1-Score with {num_participants} participants: {best_f1:.4f}')



Running VFL with seed 42...
Running VFL with 2 participants...
Best Accuracy with 2 participants: 65.73%
Best F1-Score with 2 participants: 0.5431
Running VFL with 3 participants...
Best Accuracy with 3 participants: 71.35%
Best F1-Score with 3 participants: 0.6694
Running VFL with 4 participants...
Best Accuracy with 4 participants: 81.46%
Best F1-Score with 4 participants: 0.8042
Running VFL with 5 participants...
Best Accuracy with 5 participants: 77.53%
Best F1-Score with 5 participants: 0.7543
Running VFL with 6 participants...
Best Accuracy with 6 participants: 61.24%
Best F1-Score with 6 participants: 0.3810
Running VFL with 7 participants...
Best Accuracy with 7 participants: 81.46%
Best F1-Score with 7 participants: 0.8096
Running VFL with 8 participants...
Best Accuracy with 8 participants: 74.72%
Best F1-Score with 8 participants: 0.7382
Running VFL with 9 participants...
Best Accuracy with 9 participants: 74.72%
Best F1-Score with 9 participants: 0.7282

Running VFL with s

In [ ]:
#Prototype 2 with 5 seeds

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
import random
from sklearn.utils.class_weight import compute_class_weight

# For random seeds-
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Preprocessing Titanic dataset (same as before)
def load_titanic_data(num_participants):
    def _bin_age(age_series):
        bins = [-np.inf, 10, 40, np.inf]
        labels = ["Child", "Adult", "Elderly"]
        return pd.cut(age_series, bins=bins, labels=labels, right=True).astype(str).replace("nan", "Unknown")

    def _extract_title(name_series):
        titles = name_series.str.extract(" ([A-Za-z]+)\.", expand=False)
        rare_titles = {
            "Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Dona"
        }
        titles = titles.replace(list(rare_titles), "Rare")
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        return titles

    def _create_features(df):
        df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
        df["Age"] = _bin_age(df["Age"])
        df["Cabin"] = df["Cabin"].str[0].fillna("Unknown")
        df["Title"] = _extract_title(df["Name"])
        df.drop(columns=["PassengerId", "Name", "Ticket"], inplace=True)
        return df

    def vertical_partition_rotating(df, num_participants):
        partitions = [[] for _ in range(num_participants)]
        num_features = df.shape[1]

        for i, feature in enumerate(df.columns):
            participant = i % num_participants
            partitions[participant].append(feature)

        partitioned_dfs = [pd.get_dummies(df[features]) for features in partitions]
        return partitioned_dfs


    def align_train_test_partitions(train_partitions, test_partitions):
        for i in range(len(train_partitions)):
            train_partitions[i] = pd.get_dummies(train_partitions[i], drop_first=True)
            test_partitions[i] = pd.get_dummies(test_partitions[i], drop_first=True)
            train_partitions[i], test_partitions[i] = train_partitions[i].align(test_partitions[i], join='left', axis=1, fill_value=0)
        return train_partitions, test_partitions

    def get_partitions_and_label():
        df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
        processed_df = df.dropna(subset=["Embarked", "Fare"]).copy()
        processed_df = _create_features(processed_df)
        labels = processed_df["Survived"].values

        train_df, test_df, y_train, y_test = train_test_split(processed_df, labels, test_size=0.2, random_state=42)

        train_partitions = vertical_partition_rotating(train_df.drop(columns=["Survived"]), num_participants)
        test_partitions = vertical_partition_rotating(test_df.drop(columns=["Survived"]), num_participants)

        for i in range(len(train_partitions)):
            train_partitions[i]['Survived'] = y_train
            test_partitions[i]['Survived'] = y_test

        train_partitions, test_partitions = align_train_test_partitions(train_partitions, test_partitions)

        return train_partitions, test_partitions, y_train, y_test

    train_partitions, test_partitions, y_train, y_test = get_partitions_and_label()

    def create_tensor_datasets(partitions):
        tensor_partitions = []
        for partition in partitions:
            partition = partition.apply(pd.to_numeric, errors='coerce')
            partition = partition.fillna(0)

            for col in partition.select_dtypes(include=['bool']).columns:
                partition[col] = partition[col].astype(int)

            features = partition.drop(columns=["Survived"]).values
            labels = partition["Survived"].values.astype(np.int64)

            tensor_partition = TensorDataset(torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long))
            tensor_partitions.append(tensor_partition)
        return tensor_partitions

    train_tensor_partitions = create_tensor_datasets(train_partitions)
    test_tensor_partitions = create_tensor_datasets(test_partitions)

    return train_tensor_partitions, test_tensor_partitions, train_partitions, y_train, y_test


# New Model Class
class GlobalModel(nn.Module):
    def __init__(self, input_sizes, hidden_sizes, output_size):
        super(GlobalModel, self).__init__()
        self.segments = nn.ModuleList()
        for input_size, hidden_size in zip(input_sizes, hidden_sizes):
            layers = [nn.Linear(input_size, hidden_size), nn.ReLU()]
            layers.append(nn.Linear(hidden_size, output_size))  # Each participant has its own output layer
            self.segments.append(nn.Sequential(*layers))

    def forward(self, x, active_segments):
        segment_outputs = []
        hidden_outputs = []  # Store hidden layer outputs for sharing
        start_index = 0
        for i, segment in enumerate(self.segments):
            end_index = start_index + input_sizes[i]
            if i in active_segments:
                segment_input = x[:, start_index:end_index]
                hidden_output = segment[:2](segment_input)  # Pass through hidden layer
                final_output = segment[2](hidden_output)    # Get final output after applying last layers
                hidden_outputs.append(hidden_output)
                segment_outputs.append(final_output)
            else:
                hidden_outputs.append(torch.zeros(x.size(0), hidden_sizes[i], device=x.device))
                segment_outputs.append(torch.zeros(x.size(0), output_size, device=x.device))
            start_index = end_index
        combined_output = torch.mean(torch.stack(segment_outputs), dim=0)  # Average the outputs from all participants
        return combined_output, hidden_outputs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training function
def train(model, device, train_loader, optimizer, epoch, input_sizes, participant_id, hidden_outputs_shared):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)  # Flatten input
        padded_data = torch.zeros(data.size(0), sum(input_sizes)).to(device)
        start_index = sum(input_sizes[:participant_id])
        end_index = start_index + input_sizes[participant_id]
        padded_data[:, start_index:end_index] = data
        optimizer.zero_grad()
        output, hidden_outputs = model(padded_data, active_segments=[participant_id])


        if hidden_outputs_shared is not None:
            for layer_id, hidden_output in enumerate(hidden_outputs):
                if layer_id != participant_id:
                    hidden_outputs[layer_id] = hidden_outputs_shared[layer_id]

        loss = nn.CrossEntropyLoss(weight=class_weights)(output, target)
        loss.backward()
        optimizer.step()

        hidden_outputs_shared[:] = hidden_outputs  # Update shared hidden outputs

# Gradients exchange and hidden layer outputs sahring
def selective_exchange_gradients_and_hidden(models, input_sizes, hidden_sizes):
    num_models = len(models)
    param_indices = [0]
    cumulative_index = 0
    for i in range(len(hidden_sizes)):
        cumulative_index += (input_sizes[i] * hidden_sizes[i]) + hidden_sizes[i]
        param_indices.append(cumulative_index)

    for seg in range(len(hidden_sizes)):
        start = param_indices[seg]
        end = param_indices[seg + 1]
        for param_idx in range(start, end):
            grads = []
            for model in models:
                model_params = list(model.parameters())
                if param_idx < len(model_params) and model_params[param_idx].grad is not None:
                    grads.append(model_params[param_idx].grad)
            if grads:
                avg_grad = torch.stack(grads).mean(dim=0)
                for model in models:
                    model_params = list(model.parameters())
                    if param_idx < len(model_params):
                        model_params[param_idx].grad = avg_grad.clone()

    # Hidden output exchange
    hidden_outputs_shared = [None for _ in hidden_sizes]
    for model in models:
        _, hidden_outputs = model.forward(torch.zeros(1, sum(input_sizes)), active_segments=list(range(len(hidden_sizes))))
        for layer_id, hidden_output in enumerate(hidden_outputs):
            if hidden_outputs_shared[layer_id] is None:
                hidden_outputs_shared[layer_id] = hidden_output.clone()
            else:
                hidden_outputs_shared[layer_id] += hidden_output.clone()
        for layer_id in range(len(hidden_outputs_shared)):
            hidden_outputs_shared[layer_id] /= num_models
    return hidden_outputs_shared

# Evaluation function
def evaluate(models, device, test_loaders, input_sizes):
    for model in models:
        model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch_data in zip(*test_loaders):
            data_list = []
            target_list = []
            for participant_id, (data, target) in enumerate(batch_data):
                data_list.append(data)
                target_list.append(target)

            target = target_list[0].to(device)
            for t in target_list:
                assert torch.equal(t, target), "Targets are not consistent across participants"

            data_combined = torch.cat(data_list, dim=1).to(device)

            padded_data = torch.zeros(data_combined.size(0), sum(input_sizes)).to(device)
            start_index = 0
            for participant_id in range(len(input_sizes)):
                end_index = start_index + input_sizes[participant_id]
                padded_data[:, start_index:end_index] = data_combined[:, start_index:end_index]
                start_index = end_index

            outputs = torch.zeros(data_combined.size(0), 2, device=device)
            for model in models:
                output, _ = model(padded_data, active_segments=list(range(len(model.segments))))
                outputs += output
            outputs /= len(models)

            pred = outputs.argmax(dim=1, keepdim=True)
            all_preds.extend(pred.view(-1).cpu().numpy())
            all_targets.extend(target.view(-1).cpu().numpy())

    accuracy = np.mean(np.array(all_preds) == np.array(all_targets))
    f1 = f1_score(all_targets, all_preds, average='macro')
    return accuracy, f1

# Running the experiment for multiple seeds
federated_rounds = 1000
epochs_per_round = 1
hidden_size = 10  # Single hidden layer

seeds = [42, 55, 77, 101, 123]
final_accuracies = []
final_f1_scores = []

for seed in seeds:
    set_random_seed(seed)
    print(f"\nRunning VFL with seed {seed}...")

    avg_accuracy = 0
    avg_f1 = 0

    for num_participants in range(2, 10):
        print(f"Running VFL with {num_participants} participants...")

        train_tensor_partitions, test_tensor_partitions, feature_partitions, y_train, y_test = load_titanic_data(num_participants)

        input_sizes = [partition.shape[1] - 1 for partition in feature_partitions]
        hidden_sizes = [10] * num_participants
        output_size = 2

        models = [GlobalModel(input_sizes, hidden_sizes, output_size).to(device) for _ in range(num_participants)]
        optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
        class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

        best_accuracy = 0
        best_f1 = 0

        hidden_outputs_shared = [None for _ in hidden_sizes]

        for federated_round in range(federated_rounds):
            #print(f"Federated Round {federated_round + 1}/{federated_rounds}")
            for participant_id in range(num_participants):
                train_loader = DataLoader(train_tensor_partitions[participant_id], batch_size=64, shuffle=True)
                for epoch in range(1, epochs_per_round + 1):
                    train(models[participant_id], device, train_loader, optimizers[participant_id], epoch, input_sizes, participant_id, hidden_outputs_shared)

            hidden_outputs_shared = selective_exchange_gradients_and_hidden(models, input_sizes, hidden_sizes)

            test_loaders = [DataLoader(test_tensor_partitions[i], batch_size=32, shuffle=False) for i in range(num_participants)]
            accuracy, f1 = evaluate(models, device, test_loaders, input_sizes)

            if accuracy > best_accuracy:
                best_accuracy = accuracy
            if f1 > best_f1:
                best_f1 = f1

        print(f'Best Accuracy with {num_participants} participants: {best_accuracy:.2f}%')
        print(f'Best F1-Score with {num_participants} participants: {best_f1:.4f}')



Running VFL with seed 42...
Running VFL with 2 participants...
Best Accuracy with 2 participants: 0.76%
Best F1-Score with 2 participants: 0.7604
Running VFL with 3 participants...
Best Accuracy with 3 participants: 0.82%
Best F1-Score with 3 participants: 0.8142
Running VFL with 4 participants...
Best Accuracy with 4 participants: 0.81%
Best F1-Score with 4 participants: 0.7988
Running VFL with 5 participants...
Best Accuracy with 5 participants: 0.83%
Best F1-Score with 5 participants: 0.8211
Running VFL with 6 participants...
Best Accuracy with 6 participants: 0.81%
Best F1-Score with 6 participants: 0.8088
Running VFL with 7 participants...
Best Accuracy with 7 participants: 0.81%
Best F1-Score with 7 participants: 0.8034
Running VFL with 8 participants...
Best Accuracy with 8 participants: 0.70%
Best F1-Score with 8 participants: 0.6966
Running VFL with 9 participants...
Best Accuracy with 9 participants: 0.75%
Best F1-Score with 9 participants: 0.7465

Running VFL with seed 55..

In [ ]:
# Titanic- No federation
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
import random
from sklearn.utils.class_weight import compute_class_weight

# For random seeds-
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Preprocessing Titanic dataset (same as before)
def load_titanic_data(num_participants):
    def _bin_age(age_series):
        bins = [-np.inf, 10, 40, np.inf]
        labels = ["Child", "Adult", "Elderly"]
        return pd.cut(age_series, bins=bins, labels=labels, right=True).astype(str).replace("nan", "Unknown")

    def _extract_title(name_series):
        titles = name_series.str.extract(" ([A-Za-z]+)\.", expand=False)
        rare_titles = {
            "Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Dona"
        }
        titles = titles.replace(list(rare_titles), "Rare")
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        return titles

    def _create_features(df):
        df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
        df["Age"] = _bin_age(df["Age"])
        df["Cabin"] = df["Cabin"].str[0].fillna("Unknown")
        df["Title"] = _extract_title(df["Name"])
        df.drop(columns=["PassengerId", "Name", "Ticket"], inplace=True)
        return df

    def vertical_partition_rotating(df, num_participants):
        partitions = [[] for _ in range(num_participants)]
        num_features = df.shape[1]

        for i, feature in enumerate(df.columns):
            participant = i % num_participants
            partitions[participant].append(feature)

        partitioned_dfs = [pd.get_dummies(df[features]) for features in partitions]
        return partitioned_dfs


    def align_train_test_partitions(train_partitions, test_partitions):
        for i in range(len(train_partitions)):
            train_partitions[i] = pd.get_dummies(train_partitions[i], drop_first=True)
            test_partitions[i] = pd.get_dummies(test_partitions[i], drop_first=True)
            train_partitions[i], test_partitions[i] = train_partitions[i].align(test_partitions[i], join='left', axis=1, fill_value=0)
        return train_partitions, test_partitions

    def get_partitions_and_label():
        df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
        processed_df = df.dropna(subset=["Embarked", "Fare"]).copy()
        processed_df = _create_features(processed_df)
        labels = processed_df["Survived"].values

        train_df, test_df, y_train, y_test = train_test_split(processed_df, labels, test_size=0.2, random_state=42)

        train_partitions = vertical_partition_rotating(train_df.drop(columns=["Survived"]), num_participants)
        test_partitions = vertical_partition_rotating(test_df.drop(columns=["Survived"]), num_participants)

        for i in range(len(train_partitions)):
            train_partitions[i]['Survived'] = y_train
            test_partitions[i]['Survived'] = y_test

        train_partitions, test_partitions = align_train_test_partitions(train_partitions, test_partitions)

        return train_partitions, test_partitions, y_train, y_test

    train_partitions, test_partitions, y_train, y_test = get_partitions_and_label()

    def create_tensor_datasets(partitions):
        tensor_partitions = []
        for partition in partitions:
            partition = partition.apply(pd.to_numeric, errors='coerce')
            partition = partition.fillna(0)

            for col in partition.select_dtypes(include=['bool']).columns:
                partition[col] = partition[col].astype(int)

            features = partition.drop(columns=["Survived"]).values
            labels = partition["Survived"].values.astype(np.int64)

            tensor_partition = TensorDataset(torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.long))
            tensor_partitions.append(tensor_partition)
        return tensor_partitions

    train_tensor_partitions = create_tensor_datasets(train_partitions)
    test_tensor_partitions = create_tensor_datasets(test_partitions)

    return train_tensor_partitions, test_tensor_partitions, train_partitions, y_train, y_test


class GlobalModel(nn.Module):
    def __init__(self, input_sizes, hidden_sizes, output_size):
        super(GlobalModel, self).__init__()
        self.segments = nn.ModuleList()
        for input_size, hidden_size in zip(input_sizes, hidden_sizes):
            layers = [nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, output_size)]
            self.segments.append(nn.Sequential(*layers))

    def forward(self, x, active_segments):
        segment_outputs = []
        start_index = 0
        for i, segment in enumerate(self.segments):
            end_index = start_index + input_sizes[i]
            if i in active_segments:
                segment_input = x[:, start_index:end_index]
                segment_output = segment(segment_input)
                segment_outputs.append(segment_output)
            else:
                segment_outputs.append(torch.zeros(x.size(0), output_size, device=x.device))
            start_index = end_index
        combined_output = torch.mean(torch.stack(segment_outputs), dim=0)
        return combined_output  # Only return the combined output

# Training function (here training is independent with zero-padding)
def train_independent(model, device, train_loader, optimizer, epoch, input_sizes, participant_id):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        padded_data = torch.zeros(data.size(0), sum(input_sizes)).to(device)
        start_index = sum(input_sizes[:participant_id])
        end_index = start_index + input_sizes[participant_id]
        padded_data[:, start_index:end_index] = data
        optimizer.zero_grad()
        output = model(padded_data, active_segments=[participant_id])
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluation function (evaluating independently for each participant)
def evaluate_independent(model, device, test_loader, input_sizes, participant_id):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            padded_data = torch.zeros(data.size(0), sum(input_sizes)).to(device)  # Create zero-padded input
            start_index = sum(input_sizes[:participant_id])
            end_index = start_index + input_sizes[participant_id]
            padded_data[:, start_index:end_index] = data  # Insert the active participant's data
            output = model(padded_data, active_segments=[participant_id])  # Only return the output
            pred = output.argmax(dim=1, keepdim=True)
            all_preds.extend(pred.view(-1).cpu().numpy())
            all_targets.extend(target.view(-1).cpu().numpy())

    accuracy = np.mean(np.array(all_preds) == np.array(all_targets))
    f1 = f1_score(all_targets, all_preds, average='macro')
    return accuracy, f1

# Running the experiment for multiple seeds (no collaboration)
federated_rounds = 1000
epochs_per_round = 1
hidden_size = 10  # Single hidden layer

seeds = [1,2,3,4,5]  # List of 5 different seeds
final_accuracies = []
final_f1_scores = []

for seed in seeds:
    set_random_seed(seed)
    print(f"\nRunning independent training, seed {seed}...")

    avg_accuracy = 0
    avg_f1 = 0

    for num_participants in range(2, 10):
        print(f"Running with {num_participants} independent participants...")

        train_tensor_partitions, test_tensor_partitions, feature_partitions, y_train, y_test = load_titanic_data(num_participants)

        input_sizes = [partition.shape[1] - 1 for partition in feature_partitions]  # Minus 1 for the 'Survived' label
        output_size = 2


        models = [GlobalModel(input_sizes, [hidden_size] * num_participants, output_size).to(device) for _ in range(num_participants)]
        optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]


        for participant_id in range(num_participants):
            print(f"Training participant {participant_id + 1} independently...")
            train_loader = DataLoader(train_tensor_partitions[participant_id], batch_size=64, shuffle=True)
            test_loader = DataLoader(test_tensor_partitions[participant_id], batch_size=32, shuffle=False)


            for epoch in range(1, epochs_per_round + 1):
                train_independent(models[participant_id], device, train_loader, optimizers[participant_id], epoch, input_sizes, participant_id)

            # Local model evaluation
            accuracy, f1 = evaluate_independent(models[participant_id], device, test_loader, input_sizes, participant_id)
            print(f'Participant {participant_id + 1}: Accuracy: {accuracy:.2f}, F1-Score: {f1:.4f}')
            avg_accuracy += accuracy
            avg_f1 += f1

        avg_accuracy /= num_participants
        avg_f1 /= num_participants




Running independent training, seed 1...
Running with 2 independent participants...
Training participant 1 independently...
Participant 1: Accuracy: 0.62, F1-Score: 0.4312
Training participant 2 independently...
Participant 2: Accuracy: 0.69, F1-Score: 0.6800
Running with 3 independent participants...
Training participant 1 independently...
Participant 1: Accuracy: 0.61, F1-Score: 0.3798
Training participant 2 independently...
Participant 2: Accuracy: 0.61, F1-Score: 0.3798
Training participant 3 independently...
Participant 3: Accuracy: 0.67, F1-Score: 0.6258
Running with 4 independent participants...
Training participant 1 independently...
Participant 1: Accuracy: 0.61, F1-Score: 0.3798
Training participant 2 independently...
Participant 2: Accuracy: 0.60, F1-Score: 0.5903
Training participant 3 independently...
Participant 3: Accuracy: 0.61, F1-Score: 0.3798
Training participant 4 independently...
Participant 4: Accuracy: 0.61, F1-Score: 0.3798
Running with 5 independent participant